# Transformer for neuron machine translation
Ref: https://andrewpeng.dev/transformer-pytorch/  
Ref: https://spaces.ac.cn/archives/6933  
Ref: https://github.com/graykode/nlp-tutorial  

In [1]:
import torch
from torchtext.datasets import Multi30k
from torchtext.data import BucketIterator, Field
import torch.optim as optim
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter

import os
import time
import copy
from torchsummaryX import summary

import model
import utils

## Loading data

In [2]:
BSZ = 64
SRC = Field(tokenize='spacy',
            tokenizer_language='de',
            init_token='<sos>',
            eos_token='<eos>',
            lower=True)

TRG = Field(tokenize='spacy',
            tokenizer_language='en',
            init_token='<sos>',
            eos_token='<eos>',
            lower=True)

train_data, val_data, test_data = Multi30k.splits(
    exts=('.de', '.en'), fields=(SRC, TRG))
train_iter, val_iter, test_iter = BucketIterator.splits(
    (train_data, val_data, test_data), batch_size=BSZ, shuffle = True)


SRC.build_vocab(train_data, min_freq=4)
TRG.build_vocab(train_data, min_freq=4)

In [3]:
# batch = next(iter(train_iter))

# for batch_idx, batch in enumerate(train_iter):
#     src = batch.src.transpose(0,1)[0:2]
#     trg = batch.trg.transpose(0,1)[0:2]
#     src = [' '.join(utils.itos(idx_seq, SRC)) for idx_seq in src]
#     trg = [' '.join(utils.itos(idx_seq, TRG)) for idx_seq in trg]
#     print(src)
#     print(trg)
#     if batch_idx == 0:
#         break

# print(len(train_iter))
# print(len(val_iter))
# print(len(test_iter))

# print(len(TRG.vocab))
# print(TRG.vocab.stoi[' '])
# print(TRG.vocab.itos[0])

# print(SRC.vocab.stoi['<sos>'])
# print(TRG.vocab.stoi['<sos>'])

## Define model

In [4]:
D_MODEL = 512
N_HEAD = 8
NUM_ENC_LAYERS = 6
NUM_DEC_LAYERS = 6
DIM_FEEDWORD = 2048
DROPOUT = 0.1
ACTIVATION = 'relu'
N_EPOCH = 120
LR = 0.0001

device = torch.device('cuda:2' if torch.cuda.is_available() else 'cpu')
net = model.Transformer(device, len(SRC.vocab), len(TRG.vocab), D_MODEL, N_HEAD, NUM_ENC_LAYERS,
                        NUM_ENC_LAYERS, DIM_FEEDWORD, DROPOUT, ACTIVATION).to(device)
optimizer = optim.Adam(net.parameters(), lr=LR)
criterion = nn.CrossEntropyLoss(ignore_index=SRC.vocab.stoi['<pad>'])
path = f'bsz:{BSZ}-lr:{LR}-epoch:{N_EPOCH}-d_model:{D_MODEL}-nhead:{N_HEAD}-nlayer:{NUM_ENC_LAYERS}\
-nhid:{DIM_FEEDWORD}-activation:{ACTIVATION}'
writer = SummaryWriter(os.path.join('log/', path))

best_val_bleu = 0.0
best_val_model = copy.deepcopy(net.state_dict())

In [5]:
# print(net)
# summary(net, torch.zeros((10,1), dtype = torch.long), torch.zeros((10,1), dtype = torch.long))

## Train and evaluate

In [6]:
for epoch in range(N_EPOCH):
    model.train(net, train_iter, criterion, optimizer, TRG, epoch, writer, device)
    val_loss, val_bleu = model.evaluate(net, val_iter, criterion, TRG, device)
    print(f'epoch: {epoch} | val loss: {val_loss:.3f} | val bleu: {val_bleu: .3f}')
    
    if val_bleu > best_val_bleu:
        best_val_bleu = val_bleu
        best_val_model = copy.deepcopy(net.state_dict())

## Test

In [7]:
net.load_state_dict(best_val_model)
test_loss, test_bleu = model.test(net, test_iter, criterion, TRG, device)
print(f'test loss: {test_loss:.3f} | test bleu: {test_bleu: .3f}')

## Load model

In [9]:
net.load_state_dict(torch.load('save/best_model.pth'))
test_loss, test_bleu = model.test(net, test_iter, criterion, TRG, device)
print(f'test loss: {test_loss:.3f} | test bleu: {test_bleu: .3f}')

test loss: 6.101 | test bleu:  0.377
